In [1]:
# process all of the t1 grooming data by concatenating the bouts from
# all days for which we have data, assign unique bout numbers
import os 
import numpy as np 
import pandas as pd

In [ ]:
def extract_grooming_bouts(path):
    
    angles_path = os.path.join(path, 'angles_fictrac.csv')
    pose3d_path = os.path.join(path, 'pose_3d_fictrac.csv')
    labels_path = os.path.join(os.path.dirname(path), 'behavior_labels.csv')
    
    angles = pd.read_csv(angles_path, engine = 'python', sep = None)
    pose3d = pd.read_csv(pose3d_path, engine = 'python', sep = None)
    labels = pd.read_csv(labels_path, engine = 'python', sep = None)
    
    angles_merged = pd.merge(angles, labels, how='outer')
    pose3d_merged = pd.merge(pose3d, labels, how='outer') 
    
    angles_grooming = angles_merged[angles_merged.behavior == 'T1_grooming']
    pose3d_grooming = pose3d_merged[pose3d_merged.behavior == 'T1_grooming']
    angles_grooming.to_csv(os.path.join(path, 'angles_fictrac_grooming.csv'))
    pose3d_grooming.to_csv(os.path.join(path, 'pose_3d_fictrac_grooming.csv'))
    
# assign a unique bout number to each bout (previously had duplicates due
# to running experiments on different days)
def adjust_bout_numbers(angles, pose3d, cumulative_bouts):
    
    bout_numbers = np.unique(angles.behavior_bout)
    bout_numbers_new = np.arange(cumulative_bouts, len(bout_numbers), 1)
    cumulative_bouts = cumulative_bouts + len(bout_numbers)
    
    for j in range(len(bout_numbers)):
        
        angles['behavior_bout'].replace({bout_numbers[j]:bout_numbers_new[j]}, inplace=True)
        pose3d['behavior_bout'].replace({bout_numbers[j]:bout_numbers_new[j]}, inplace=True)
    
    return angles, pose3d, cumulative_bouts


# removes grooming bouts from dataset that are less than a specified 
# number of frames (too short to analyze)
def remove_short_bouts(angles, pose3d, min_frames):    
    
    bout_numbers = np.unique(angles.behavior_bout)
    bout_lengths = np.zeros(bout_numbers.shape)
    for j in range(len(bout_numbers)):
        bout_lengths[j] = len(angles[angles.behavior_bout == bout_numbers[j]])
    
    saved_bouts = bout_lengths[bout_lengths >= min_frames]
    angles_new = angles[output_angles.behavior_bout.isin(saved_bouts)]
    pose3d_new = pose3d[output_pose3d.behavior_bout.isin(saved_bouts)]
    
    return angles_new, pose3d_new

In [ ]:
data_path = r'/media/turritopsis/katie/grooming/t1-grooming/data
save_path = r'/media/turritopsis/katie/grooming/t1-grooming/data'
out_path_angles = os.path.join(save_path, 'grooming_combined_angles.csv')
out_path_pose3d = os.path.join(save_path, 'grooming_combined_pose3d.csv')

In [ ]:
# combine all grooming bouts, assign a unique bout number, and save
# angles and pose3d to separate csv files
(root, dirs, files) = next(os.walk(data_path))
dirs = sorted(dirs)
cumulative_bouts = 0
output_angles = pd.DataFrame()
output_pose3d = pd.DataFrame()

for directory in dirs:
    folder = os.path.join(path, directory)#, 'summaries')
    # extract_grooming_bouts(folder)
    angles = pd.read_csv(os.path.join(folder, 'angles_fictrac_grooming.csv'), engine = 'python', sep = None, index_col = False)
    pose3d = pd.read_csv(os.path.join(folder, 'pose_3d_fictrac_grooming.csv'), engine = 'python', sep = None, index_col = False)
    angles, pose3d, cumulative_bouts = adjust_bout_numbers(angles, pose3d, cumulative_bouts)
    output_angles = output_angles.append(angles)
    output_pose3d = output_pose3d.append(pose3d)
     
output_angles.to_csv(out_path_angles)
output_pose3d.to_csv(out_path_pose3d)

In [ ]:
# removes grooming bouts from dataset that are less than a 
# specified number of frames (too short to analyze)
min_frames = 100
angles = pd.read_csv(out_path_angles, index_col = False)
pose3d = pd.read_csv(out_path_pose3d, index_col = False)
angles_new, pose3d_new = remove_short_bouts(angles, pose3d, min_frames)
angles_new.to_csv(os.path.join(path, 'grooming_combined_cleaned_angles.csv'))
pose3d_new.to_csv(os.path.join(path, 'grooming_combined_cleaned_pose3d.csv'))